# **Authors**: Max Schuman, Alex Meci, Kyle Smilon
# Hand Gesture Calculator

In [413]:
import cv2
import numpy as np
import math
from skimage import color


In [414]:
# Create Global Variables
background1 = None
background2 = None
# roi = region of interest
roi_top = 200
roi_bottom = 450
roi_right = 370
roi_left = 620
ratio = 0 
expressions = []
text = ''
operators = ['+','-','*','/']
answer = None
thumb = False
num_text = '0'
new_op = ''
old_op = ''

In [415]:
def create_text(frame):
    """
    Arguments:
    frame: The frame of the video 

    This function creates the text for all the 
    operations placed on the screen.
    """
    cv2.putText(frame,'+',(30,50), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255),2)
    cv2.putText(frame,'-',(30,120), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255),2)
    cv2.putText(frame,'*',(30,200), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255),2)
    cv2.putText(frame,'/',(30,280), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255),2)
    cv2.putText(frame,'=',(30,350), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255),2)
    cv2.putText(frame,'C ',(30,450), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255),2)

In [416]:
# Create the Clear Function
def clear():
    """
    This acts as the clear function on a calculator
    reseting all the variables
    """
    global expressions, text, num_text, new_op, old_op, answer
    expressions = []
    text = ''
    num_text = '0'
    new_op = ''
    old_op = ''
    answer = None


In [417]:
def background_weight1(frame):
    """
    Arguments:
    frame: the current frame being captured
    
    This takes the weighted average of an images picture.
    It is used to help figure out what is part of the original background.
    Used on the rectangles background.

    returns: None 
    """
    global background1
    if background1 is None:
        background1 = frame.copy().astype('float')

    
    cv2.accumulateWeighted(frame,background1,.5)

In [418]:

def background_weight2(frame):
    """
    Arguments:
    frame: the current frame being captured
    
    This takes the weighted average of an images picture.
    It is used to help figure out what is part of the original background.
    Used on the opperators background.

    returns: None 
    """
    global background2
    if background2 is None:
        background2 = frame.copy().astype('float')

    
    cv2.accumulateWeighted(frame,background2,.5)

In [419]:

def segment(frame,background,threshold_min = 30):
    """
    Arguments:
    frame: The Box frame
    background: background of the image
    threshhold_min: The minimum threshold needed
    
    This function takes the frame and background and begins to
    threshold them. It allows us to find the difference between the background 
    and begin to "read" the hand gestures. The background will find 
    issues if the lighting is not good.

    returns: None if there are no contours found/thresholded background and hand segment 
    """
    diff = cv2.absdiff(background.astype('uint8'),frame)
    ___,thresh = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    kernel = np.ones((3,3),np.uint8)
    thresholded = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)
    contours,__ = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    # If there are no contours then there is no hand
    if len(contours) == 0:
        return None
    else:
        # Only use the max contours 
        hand_segment = max(contours,key=cv2.contourArea)
        return (thresholded,hand_segment)

In [420]:
def calculate_math(defects, approx):
    """
    Arguments:
    defects: list of the convexity defects of a contour(the hand contour)
    
    This function calculates the main math behind finding the contours of the hand.
    Using the defects which has a start, end, farthest, and approx dist to farthest. 
    It then can calculate each side of the triangle. This information is then used in
    the calculation of the angles using arc cosine
    """
    global thumb, fingies
    for i in range(defects.shape[0]):
        # start, end, farthest, approx dist to farthest point
        s,e,f,d = defects[i,0]
        start = tuple(approx[s][0])
        end = tuple(approx[e][0])
        far = tuple(approx[f][0])

        # find length of all sides of triangle
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        s = (a+b+c)/2
        ar = math.sqrt(s*(s-a)*(s-b)*(s-c))

        #distance between point and convex hull
        d=(2*ar)/a

        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57

        # Works by ignoring angles greater than 90 and anything that could be noise
        if angle <= 100 and d>30:
            fingies += 1
            cv2.circle(roi, far, 3, (255,0,0), -1)
            if(angle>50):
                thumb = True
        
        cv2.line(roi,start, end, (0,255,0), 2)    

In [421]:
def finger_count(fingies, thumb):
    """
    Arguments:
    fingies: The number of fingers being shown
    thumb: Boolean value to decide if the user wants numbers >5

    This function processes the finger count. Using the collection of the angle and distance
    from the calculate_math() funtion we can get an accurate relation to the amount
    of fingers that are being used. Then using the thumb argument the program can decide which number the 
    user is attempting to use.
    """
    global first_input
    if(num_frames%50 == 0):

        if(fingies==1):
            if(ratio<12 and (first_input == False)):
                cv2.putText(frame_copy,'0',(80,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text= '0'
                expressions.append(num_text)
            elif(ratio<17.5):
                cv2.putText(frame_copy,'6',(80,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text= '6'
                expressions.append(num_text)
            else:
                first_input = False
                cv2.putText(frame_copy,'1',(80,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text= '1'
                expressions.append(num_text)


        if(fingies==2):
            first_input = False
            if(thumb == True):
                cv2.putText(frame_copy,'7',(80,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '7'
            else:   
                cv2.putText(frame_copy,'2',(80,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '2'
            expressions.append(num_text)

        if(fingies==3):
            first_input = False
            if(thumb == True):
                cv2.putText(frame_copy,'8',(80,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '8'
            else:
                cv2.putText(frame_copy,'3',(80,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)  
                num_text = '3'
            expressions.append(num_text)

        if(fingies==4):
            first_input = False
            if(thumb == True):
                cv2.putText(frame_copy,'9',(80,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '9'
            else:
                cv2.putText(frame_copy,'4',(80,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '4'
            expressions.append(num_text)

        if(fingies==5):
            first_input = False
            cv2.putText(frame_copy,'5',(300,40), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            num_text = '5'
            expressions.append(num_text)

In [422]:
def expression_selection(expressions, old_op, text, roi2):
    """
    Arguments:
    expressions: list of all the expressions "+, -, /, *, ="
    old_op: The old operation that was previously chosen 
    text: The calculator text that shows current operation and numbers

    Function allows the user to select the operation that they would
    like to perform. Using the finger_segment we can locate the users
    finger that is outside of the box and update x and y coordinates. 
    The x and y are used to diside which operation the user is selecting.
    It checks the frames along with the previous expression to make sure that it does
    not duplicate.

    return: This returns the "calculator" text
    """
    global first_input, new_op, num_text, answer, finger 
    __ , finger_segment = finger
    conv_hull2 = cv2.convexHull(finger_segment)
    point = tuple(conv_hull2[conv_hull2[:, :, 1].argmin()][0])
    x = point[0] 
    y = point[1]
    cv2.circle(roi2,(x,y),3,(0,0,255), -1)
    
    if((len(expressions)!=0) and (expressions[-1] not in operators)):
        if((x>0 and x<=30) and (y>0 and y<50)):
            cv2.putText(frame_copy,'+',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '+'
            if((old_op != new_op) and (len(expressions)!=0)):
                expressions.append(new_op)
        if((x>0 and x<=30) and (y>80 and y<120)):
            cv2.putText(frame_copy,'-',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '-'
            if((old_op != new_op) and len(expressions)!=0):
                expressions.append(new_op)
        if((x>0 and x<=30) and (y>140 and y<180)):
            cv2.putText(frame_copy,'*',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '*'
            if((old_op != new_op) and len(expressions)!=0):
                expressions.append(new_op)
        if((x>0 and x<=30) and (y>200 and y<280)):
            cv2.putText(frame_copy,'/',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '/'
            if((old_op != new_op) and len(expressions)!=0):
                expressions.append(new_op)
        if((x>0 and x<=30) and (y>300 and y<340)):
            cv2.putText(frame_copy,'=',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '='
            # Python built in function, parses through information
            # then performs the calculations
            answer = eval(text)
        if((x>0 and x<=30) and (y>380 and y<420)):
            clear()

    # Update variables
    first_input = True
    old_op = new_op
    return text
            


In [423]:
def finger_count(fingies, thumb):
    """
    Arguments:
    fingies: the number of fingers that the program reads

    This function processes the finger count. Using the collection of the angle and distance
    from the do_maths() funtion we can get an accurate relation to the amount
    of fingers that are being used. Function also has the ability to see if the thumb is present.
    The thumb grants access to 6, 7, 8, 9 on the calculator.
    """
    global first_input
    if(num_frames%50 == 0):
        if(fingies==1):
            if(ratio<12 and (first_input == False)):
                cv2.putText(frame_copy,'0',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '0'
                expressions.append(num_text)
            elif(ratio<17.5):
                cv2.putText(frame_copy,'6',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '6'
                expressions.append(num_text)
            else:
                first_input = False
                cv2.putText(frame_copy,'1',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '1'
                expressions.append(num_text)


        if(fingies==2):
            first_input = False
            if(thumb == True):
                cv2.putText(frame_copy,'7',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '7'
            else:   
                cv2.putText(frame_copy,'2',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '2'
            expressions.append(num_text)

        if(fingies==3):
            first_input = False
            if(thumb == True):
                cv2.putText(frame_copy,'8',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '8'
            else:
                cv2.putText(frame_copy,'3',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)  
                num_text = '3'
            expressions.append(num_text)
        
        if(fingies==4):
            first_input = False
            if(thumb == True):
                cv2.putText(frame_copy,'9',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '9'
            else:
                cv2.putText(frame_copy,'4',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                num_text = '4'
            expressions.append(num_text)

        if(fingies==5):
            first_input = False
            cv2.putText(frame_copy,'5',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            num_text = '5'
            expressions.append(num_text)

In [424]:
cam = cv2.VideoCapture(0)
num_frames = 0
first_input = True
while True:
    __,frame = cam.read()
    frame_copy = frame.copy()
    frame_copy=cv2.flip(frame_copy,1)
    roi = frame_copy[roi_top:roi_bottom,roi_right:roi_left]

    gray_rect_roi = (color.rgb2gray(roi)*255).astype('uint8')
    gray_rect_roi = cv2.GaussianBlur(gray_rect_roi,(5,5),100)
    
    create_text(frame_copy)
    
    roi2 = frame_copy[20:500,20:140]
    # Change image to gray
    gray_roi2 = (color.rgb2gray(roi2)*255).astype('uint8')
    # Apply Blur to image
    gray_roi2 = cv2.GaussianBlur(gray_roi2,(5,5),100)

    if num_frames<200:
        # Weight of the rectangles background
        background_weight1(gray_rect_roi)
        # Weight of the opperators background
        background_weight2(gray_roi2)
        cv2.putText(frame_copy,"DOING MAGIC, DONT MOVE",(300,500),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,70),2)
        cv2.imshow('Finger Count',frame_copy)  

    else:
        hand = segment(gray_rect_roi,background1)
        if hand is not None:
            cv2.imshow('Finger Count',frame_copy)
            thresholded , hand_segment = hand
            # Find the convex hull of the hand points
            conv_hull =cv2.convexHull(hand_segment)

            # Find the area of the contour on the hand segment
            area_contour = cv2.contourArea(hand_segment)
            # Find the area of the contour on the convex hull
            area_hull = cv2.contourArea(conv_hull)
            if(area_contour != 0):
                # We take the area hull and area contour
                # These help calculate their ratio to eachother
                ratio=((area_hull-area_contour)/area_contour)*100
            
            stdev = 0.0005*cv2.arcLength(hand_segment,True)
            # More percise way to measure the verticies
            approx = cv2.approxPolyDP(hand_segment,stdev,True)
            
            conv_hull = cv2.convexHull(approx,returnPoints =False)
            defects = None
            # throughs error not sure why but this fixes it
            # using the cv2 hand contour specif "convexityDefects" we find
            # the specific defects that are needed
            try:
                defects = cv2.convexityDefects(approx,conv_hull)
            except:
                pass
            if defects is not None:
                fingies=0
                thumb = False

                calculate_math(defects, approx)
                fingies+=1
                finger_count(fingies, thumb)
                cv2.imshow('Thresholded',thresholded)
        text = ''
        for op in expressions: 
            text += op 
        else:
            cv2.imshow('Finger Count',frame_copy)
            finger = segment(gray_roi2,background2)
            # Find the the location of the finger that is clicking the expressions
            # one this is done then the finger is followed by the the point and
            # If the location of the point is within the x and y of one of the expressions we 
            # Then perform the expression that is "clicked"
            if finger is not None:
                text = expression_selection(expressions, old_op, text, roi2)    

    if(answer is not None):
        # If there is no answer yet add "=" along with answer
        cv2.putText(frame_copy,"="+str(answer),(100,175), cv2.FONT_HERSHEY_SIMPLEX,1,(70, 255, 255),2)
    cv2.putText(frame_copy,text,(100,125), cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 70),2)
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(255, 255, 70),5)
    num_frames += 1        
    cv2.imshow('Finger Count',frame_copy)
        
    k = cv2.waitKey(1) & 0xFF          
    
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 